Alexis Perumal, 5/4/20
HW22, Level 1 with help from Kevin Nguyen

In [0]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"


In [0]:
import findspark
findspark.init()

# New Section

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-05-07 07:15:39--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2020-05-07 07:15:39 (8.93 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


In [7]:
from pyspark import SparkFiles
# url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
category_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), sep="\t", header=True)
category_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [0]:
from pyspark.sql.types import IntegerType
category_df = category_df.withColumn("customer_id", category_df["customer_id"].cast(IntegerType()))
category_df = category_df.withColumn("product_parent", category_df["product_parent"].cast(IntegerType()))
category_df = category_df.withColumn("star_rating", category_df["star_rating"].cast(IntegerType()))
category_df = category_df.withColumn("helpful_votes", category_df["helpful_votes"].cast(IntegerType()))
category_df = category_df.withColumn("total_votes", category_df["total_votes"].cast(IntegerType()))

In [9]:
# Count the number of rows
category_df.count()

3105520

In [0]:
# For development purposes, limit the size of the dataframe so things go quicker!
# category_df = category_df.limit(1000)

In [11]:
# Count the number of rows
category_df.count()

3105520

In [12]:
# Drop null values
category_df = category_df.dropna()
category_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [13]:
# Count the number of rows
category_df.count()

3105360

In [14]:
# Drop duplicates
category_df = category_df.dropDuplicates(subset=['product_id'])
category_df.show()
category_df.count()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48358035|R1RLFZ9RTVUP67|0001714422|     987285316|'C' Is for Clown ...|           Books|          5|            6|          6|   N|                Y|The authors' name...|The text consists...| 2004-09-11|
|         US|   45160727|R27VM3ZQL6NICW|0001837192|     148075571|Pieter Brueghel's...|           Books|          5|    

779717

Note the schema of the four tables:

CREATE TABLE review_id_table (
  review_id TEXT PRIMARY KEY NOT NULL,
  customer_id INTEGER,
  product_id TEXT,
  product_parent INTEGER,
  review_date DATE -- this should be in the formate yyyy-mm-dd
);

-- This table will contain only unique values
CREATE TABLE products (
  product_id TEXT PRIMARY KEY NOT NULL UNIQUE,
  product_title TEXT
);

-- Customer table for first data set
CREATE TABLE customers (
  customer_id INT PRIMARY KEY NOT NULL UNIQUE,
  customer_count INT
);

-- vine table
CREATE TABLE vine_table (
  review_id TEXT PRIMARY KEY,
  star_rating INTEGER,
  helpful_votes INTEGER,
  total_votes INTEGER,
  vine TEXT
);


In [15]:
# Build the Table 1, in PySpark: review_id_table

from pyspark.sql.functions import to_date
review_id_df = category_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1RLFZ9RTVUP67|   48358035|0001714422|     987285316| 2004-09-11|
|R27VM3ZQL6NICW|   45160727|0001837192|     148075571| 2001-11-01|
|R2OKG9RRHSKFD6|   50538533|0002005565|     154179403| 2004-03-14|
|R22AZ61OGN0ZS7|   52973496|0002151731|     982420487| 2000-04-02|
|R2A17E76SOXH2J|   51787295|0002216973|     365315873| 1999-05-11|
|R3KQF191734AA5|   52988759|0002217236|     890589691| 2003-06-12|
|R3OCA2W1ALQC6B|   24036678|000255111X|     350965545| 2005-02-08|
| RDVTY4CQ6O35A|   51840199|0004708237|     124594219| 2003-08-09|
|R2S106DYOFAHM7|   53084107|0006544150|     602126684| 2004-10-10|
|R1L7PLBT5T72T5|   26392470|000717201X|      35278896| 2005-02-09|
|R1LQ9CNWAZB3T0|   51706672|0007640617|     191873731| 2002-12-18|
| RK4MMTA1XNB5L|   51074760|0020710518|     154305790| 1999-11

In [16]:
# Build the Table 2, in PySpark: products table

# Extract products table info
products_df = category_df.select(["product_id", "product_title"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0001714422|'C' Is for Clown ...|
|0001837192|Pieter Brueghel's...|
|0002005565|             Thieves|
|0002151731|The Greatest Bene...|
|0002216973|     Red Adam's Lady|
|0002217236|       Running Blind|
|000255111X|The Adventures of...|
|0004708237|Collins Gem Shubu...|
|0006544150|           In Xanadu|
|000717201X|The Lord of the R...|
|0007640617|Energize Your Lif...|
|0020710518|Mud Soldiers: Lif...|
|002075910X|Group Marriage;: ...|
|0020960107|The Only Way to C...|
|0023605103|Parables of Jesus...|
|0025239716|The Baseball Chro...|
|0025703811|Life on the Rim: ...|
|0027182223|The CHIMES OF ALY...|
|0027701808|             Dogsong|
|0028610881|Complete Idiot's ...|
+----------+--------------------+
only showing top 20 rows



In [17]:
# Build the Table 3, in PySpark: customers table
import pyspark.sql.functions as sf

customers_df = category_df.groupBy("customer_id").agg(sf.count("product_id").alias("customer_count"))
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   24470475|             1|
|   14520818|             1|
|   45862850|             2|
|   52595452|            18|
|   44127921|            20|
|   52653406|             3|
|   44040070|             2|
|   51234656|             1|
|   32550768|             1|
|   44199256|             2|
|   52267795|             1|
|   49389861|            59|
|   50919814|             1|
|   49041853|             1|
|   53046320|             6|
|   52649371|             1|
|   14544320|             1|
|   23526456|             3|
|   50669529|           127|
|   51078326|             5|
+-----------+--------------+
only showing top 20 rows



In [18]:
# Build the Table 4, in PySpark: vine_table

vine_table_df = category_df.select(["review_id", "star_rating", "helpful_votes", "total_votes",
                                       "vine"])
vine_table_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1RLFZ9RTVUP67|          5|            6|          6|   N|
|R27VM3ZQL6NICW|          5|            0|          0|   N|
|R2OKG9RRHSKFD6|          2|            2|          3|   N|
|R22AZ61OGN0ZS7|          2|           24|         60|   N|
|R2A17E76SOXH2J|          5|            4|          4|   N|
|R3KQF191734AA5|          5|            3|          3|   N|
|R3OCA2W1ALQC6B|          5|            2|          2|   N|
| RDVTY4CQ6O35A|          4|            3|          3|   N|
|R2S106DYOFAHM7|          5|           15|         15|   N|
|R1L7PLBT5T72T5|          5|            2|          2|   N|
|R1LQ9CNWAZB3T0|          5|            5|          5|   N|
| RK4MMTA1XNB5L|          5|            0|          0|   N|
|R2S4YWBFNZMPMR|          4|           14|         14|   N|
|R2ANVI9662SI41|          5|           2

Push to RDS

In [0]:
 # Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.ccavomop9z5f.us-east-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"postgres", 
          "password": "", 
          "driver":"org.postgresql.Driver"}

In [0]:
# Table 1
# Write DataFrame to review_id_table table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [21]:
review_id_df.head()

Row(review_id='R1RLFZ9RTVUP67', customer_id=48358035, product_id='0001714422', product_parent=987285316, review_date=datetime.date(2004, 9, 11))

In [0]:
# Table 2
# Write DataFrame to products table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [23]:
# Table 3
# Write DataFrame to customers table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

Py4JJavaError: ignored

In [0]:
# Table 4
# Write DataFrame to vine_table table in RDS
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)